# 2c. Extract time series from each ROI

**Author: Carolyn Parkinson**

**Email: <cparkinson@ucla.edu>**

**Lab website: http://csnlab.org**

**August 2018**
#### Note: Notebooks 2a, 2b and 2c are created just for the purposes of following along and possibly using as a reference in the future, since many of you won't have installed AFNI and since the input files would be very large and would take a long time to download.

After preprocessing the EPI data and concatenating pre-processed data across runs, this script:

1) Averages the preprocessed time series across voxels within anatomical ROI

2) Saves these averaged pre-processed time series for each ROI for each subject in a nested dictionary (*all_subj_roi_ts.json*) formatted like this:

*all_ts_dict[subject_id][roi_label] = [t1, t2, ..., tn]*


It takes as input:

- Each subject's preprocessed data concatenated across runs (*subjectid_epi_all_preprocessed+orig*)

- Each subject's Freesurfer parcellation and cortical segmentation
 file (*aparc+aseg_rank_Alnd_Exp+orig*), which is already in line with their functional data

- The table of values and corresponding ROI labels for that subject's FreeSurfer cortical parcellation and tissue segmentation file (*aparc+aseg_rank.niml.lt*)

- Lists of subject IDs (*fmri_subjects.json*) and ROIs in *aparc+aseg_rank.niml.lt* to exclude (*exclude_rois.json* -- e.g., white matter, ventricles, non-brain tissue)




In [2]:
import os
import subprocess
import json
import glob
import csv
import numpy as np

In [2]:
study_path = "./data"

# load in list of fMRI subjects
with open("{}/fmri/fmri_subjects.json".format(study_path)) as data_file:
    subj_list = json.load(data_file)

# load in a list of ROI labels to exclude (ventricles, white matter, etc.)
with open("{}/fmri/exclude_rois.json".format(study_path)) as data_file:
    rois_to_exclude = json.load(data_file)

In [3]:
# some convenience functions

def sh(c):
    '''
    run shell commands
    '''
    subprocess.call(c, shell = True)

    
def ld_writeDicts(filePath, dict):
    '''
    write dictionaries to files with json encoding
    '''
    f = open(filePath,'w')
    newData = json.dumps(dict, sort_keys = True, indent = 4)
    f.write(newData)
    f.close()

In [54]:
all_ts_dict = {}

for subject in subj_list:
    
    subj_dir = "{}/fmri/example_subj/{}".format(study_path, subject)
    os.chdir(subj_dir)
    
    all_epi = "{}_epi_all_preprocessed+orig".format(subject)

    # specify table of ROI labels from Freesurfer cortical parcellation
    lookup_table = "{}/aparc+aseg_rank.niml.lt".format(subj_dir)

    # create a dictionary to fill that matches ROI numbers to labels
    roi_dict = {}

    with open(lookup_table) as csvfile:
        # skip first 4 lines (header)
        next(csvfile)
        next(csvfile)
        next(csvfile)
        next(csvfile)
        reader = csv.DictReader(csvfile, delimiter = " ", 
                                fieldnames = ["roi_num", "roi_label"])
        for row in reader:
            roi_dict[row['roi_num']] = row['roi_label']

    # save as text file with json encoding; 
    # these can vary a bit from subject to subject
    ld_writeDicts('{}_roi_dict_dk_atlas.json'.format(subject), roi_dict)

    # now make masks of cortical areas from Freesurfer parcellation 
    # so that we can extract the average time series from each mask.

    # Get the total number of ROIs in this subject's parcellation file; this is
    # 1 less than the length of the lookup table due to the file footer. The
    # length of different subjects' lookup tables may vary a bit, as some
    # subjects have a 5th ventricle in the FreeSurfer subcortical segmentation;
    # see http://surfer.nmr.mgh.harvard.edu/fswiki/SubcorticalSegmentation/
    all_roi_num = range(1, (len(roi_dict)-1))

    # get Freesurfer parcellation file that's been aligned to our anatomical scan
    aparc = "aparc+aseg_rank_Alnd_Exp"

    # resample parcellation file to functional resolution
    resamp_cmd = ("3dresample -master {} -rmode NN "
                  "-input {}+orig -prefix {}_3mm").format(all_epi, aparc, aparc)
    sh(resamp_cmd)

    # dictionary to store preprocessed time series indexed by ROI
    subj_ts_dict = {}

    for roi_id in all_roi_num:
        # extract ROI label form ROI dictionary
        label_roi = roi_dict[str(roi_id)]
        
        if label_roi not in rois_to_exclude:

            # extract the average time series across voxels that ROI
            avg_cmd = ("""3dmaskave -quiet -mask {}_3mm+orig"<{}..{}>" {} > avg_{}.1D""").format(aparc, roi_id, roi_id, all_epi, label_roi)
            sh(avg_cmd)


            # then read in the 1D file generated by the above command (average time
            # series across voxels within a given ROI) and add it to a dictionary
            # indexed by ROI label
            this_vec = "avg_{}.1D".format(label_roi)
            subj_ts_dict[label_roi] = np.loadtxt(this_vec).tolist()    
    
    # move the individual ROI-wise time series 1D files to their own sub-directory   
    avg_ts_dir = "{}/avg".format(subj_dir)
    if not os.path.exists(avg_ts_dir):
        os.makedirs(avg_ts_dir)
    mv_sub_cmd = "mv avg*1D {}/".format(avg_ts_dir)
    sh(mv_sub_cmd)

    # add this subject's time series data for 80 ROIs to master dictionary
    all_ts_dict[subject] = subj_ts_dict

# save dictionary of all subjects' data for all 80 ROIs to a json file
ld_writeDicts('{}/all_subj_roi_ts.json'.format(study_path), all_ts_dict)
   